In [1]:
from bs4 import BeautifulSoup
import grequests
import requests
from multiprocessing import Pool
import numpy as np
import re

In [2]:
def strip_white_chars(text):
    return text.strip("\n").strip(" ")

In [44]:
def get_product_data(product_request_categ):
    """Function is responsible for scraping product data from 4U.am product content
    """
    product_request=product_request_categ[0]
    category = product_request_categ[1]
    try:
        soup = BeautifulSoup(product_request.content, 'html.parser')
        sp = soup.find_all("div", {"class": "info_container"})[0]

        product_data = {}
        product_data["product"] = sp.find('p', attrs={'class' : 'name_info'}).text

        brand_partner = sp.find_all('div', attrs={'class' : 'brand_partner'})
        product_data["item_code"] = brand_partner[0].span.strong.text
        product_data["store"] = strip_white_chars(brand_partner[1].find_all("span")[0].a.text)
        try:
            product_data["brand"] = strip_white_chars(brand_partner[1].find_all("span")[1].a.text)
        except:
            pass
        product_data["url"] = product_request.url
        product_data["price"] = strip_white_chars(sp.find("p", attrs={'class' : 'item_price'}).span.span.text)
        product_data["item description"] = strip_white_chars(sp.find("div", attrs={'id' : 'description'}).div.text)
        product_data["image url"] = soup.find("div", attrs={'class' : 'show_tumb'}).img["src"]
        product_data["category"] = category

        return product_data
    except:
        return dict()

In [45]:
def get_category_names():
    r = requests.get("https://4u.am/en/")
    soup = BeautifulSoup(r.content, 'html.parser')
    text = str(soup)
    return list(set(re.findall(r"\/en/category/([\w-]+)[\?|\"]\b", text)))

In [46]:
def extract_category_contents(category_names):
    rs = (grequests.get("https://4u.am/en/category/"+cat) for cat in category_names)
    return grequests.map(rs)

def extract_item_contents(item_urls):
    rs = (grequests.get("https://4u.am"+item) for item in item_urls)
    return grequests.map(rs)

In [47]:
def extract_item_urls(category_request):
    items = []
    soup = BeautifulSoup(category_request.content, 'html.parser')
    sp= soup.find_all('div', attrs={'class' : 'category_item'})
    for i in sp: items.append((i.find_all("a")[1]["href"],category_request.url.split("/")[-1]))
    return items

In [54]:
def extract_item_info(category_names):
    
    with Pool(5) as p:
        item_urls = p.map(extract_item_urls, extract_category_contents(category_names), chunksize=1)
        
    urls = [j[0] for i in item_urls for j in i]
    category = [j[1] for i in item_urls for j in i]
    item_contents = extract_item_contents(urls)

    zip_item_categ = list(zip(item_contents, category))

    with Pool(5) as p:
        result = p.map(get_product_data,zip_item_categ)
    
    return result

In [55]:
category_names =  get_category_names()

In [56]:
test = extract_item_info(category_names)

In [58]:
import pickle
with open("data.pkl", "wb") as f:
    pickle.dump(test,f)

In [111]:
with open("data.pkl", "rb") as f:
    qq = pickle.load(f)